Let me help you modify the `main()` function to test both JSON and text file loading while keeping the existing functionality. Here's the updated version:

```python:src/seed_data.py
def main():
    """
    Hàm chính để kiểm thử các chức năng của module
    Thực hiện:
        1. Test seed_milvus với dữ liệu từ file local 'stack.json'
        2. Test seed_milvus với dữ liệu từ file text 'sample.txt'
        3. (Đã comment) Test seed_milvus_live với dữ liệu từ trang web stack-ai
    Chú ý:
        - Đảm bảo Milvus server đang chạy tại localhost:19530
        - Các biến môi trường cần thiết (như OPENAI_API_KEY) đã được cấu hình
    """
    # Test với file JSON
    print("\nTesting with JSON file...")
    seed_milvus('http://localhost:19530', 'data_test', 'stack.json', 'data', use_ollama=False)
    
    # Test với file text
    print("\nTesting with text file...")
    seed_milvus('http://localhost:19530', 'fairy_tale', 'sample.txt', 'data', use_ollama=False)
    
    # Test seed_milvus_live với URL trực tiếp (commented)
    # seed_milvus_live('https://www.stack-ai.com/docs', 'http://localhost:19530', 'data_test_live', 'stack-ai', use_ollama=False)
```

The rest of the code can remain unchanged since we already have the text file handling functionality in place through:

1. File type detection:

```50:55:src/seed_data.py
    # Determine file type and load accordingly
    file_extension = filename.split('.')[-1].lower()
    if file_extension == 'txt':
        local_data, doc_name = load_text_file(filename, directory)
    else:
        local_data, doc_name = load_data_from_local(filename, directory)
```


2. Text file processing:

```155:193:src/seed_data.py
def load_text_file(filename: str, directory: str) -> list:
    """
    Load and process text file into documents
    Args:
        filename (str): Name of text file to read
        directory (str): Directory containing the file
    Returns:
        list: List of dictionaries with page_content and metadata
    """
    file_path = os.path.join(directory, filename)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        
    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    
    chunks = text_splitter.split_text(text)
    
    # Create document format matching JSON structure
    documents = []
    for i, chunk in enumerate(chunks):
        documents.append({
            'page_content': chunk,
            'metadata': {
                'source': file_path,
                'content_type': 'text/plain',
                'title': filename,
                'description': f'Part {i+1} of {filename}',
                'language': 'en',
                'start_index': i
            }
        })
    
    return documents, filename.rsplit('.', 1)[0].replace('_', ' ')
```


You can now run the script and it will:
1. Load and process the JSON data from stack.json into the 'data_test' collection
2. Load and process the text data from sample.txt into the 'fairy_tale' collection
3. Both will be stored in your Milvus database with proper chunking and metadata

To test it, simply run:
```bash
python src/seed_data.py
```